## Business Undestanding


Der erste Schritt, das Business Understanding, dreht sich um ... \
Bei der vorliegenden Projektaufgabe gilt es, drei Prognosemodelle zu erstellen, um auf Basis der vorliegenden Daten zu den Brunnen bestimmen zu können, welche der gegebenen Brunnen funktionieren, welche nicht und welche repariert werden müssen. Dabei soll anhand der ML-Pipeline, das heißt dem CRISP-DM Modell vorgegangen werden und die Güte der einzelnen Modelle anhand der AUC auf den Testdaten qualifiziert werden. 

## Data Understanding


Beim Data Understanding geht es darum, ... \
Das heißt bei der vorliegenden Projektaufgabe sollen / werden

## Data Preperation

## Modeling

## Evaluation

## Deployment